<a href="https://colab.research.google.com/github/murillo-borges/webscraping-livelo-esfera/blob/main/Webscraping_Whats_Milhas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dados padronizados

In [1]:
pip install pandas

In [2]:
pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 7.2 MB/s eta 0:00:00


In [3]:
#Importando bibliotecas
import requests
import pandas as pd
import re
import datetime
from bs4 import BeautifulSoup
from datetime import date
from zoneinfo import ZoneInfo

#Coletando a data do dia
data_hoje = date.today()
data_formatada = data_hoje.strftime("%d/%m/%Y")

# Lista de nomes que você quer capturar (normalizados)
nomes_alvo = ["livelo", "shopping livelo", "esfera", "smiles", "azul", "shopping latam", "latam"]

#Horarios de corte para envios
hora_max_grupo_beleza = 11
hora_min_grupo_esportivo = 11

#Buscando a hora atual
agora_brasilia = datetime.datetime.now(ZoneInfo("America/Sao_Paulo"))

hora_atual = int(agora_brasilia.strftime("%H"))

In [4]:
#Criando o texto de bom dia
texto_bom_dia = """
☀️ *Bom dia, milheiros!* ✈️

Antes de começar o dia, segue a nossa escala de análise das promoções para você aproveitar as melhores promoções no acúmulo de milhas:

Nós avaliamos cada promoção em 5 níveis, com base no retorno de milhas/pontos:

• *Nível 1 (Ruim) ⭐* → Sugiro aguardar algo melhor.
• *Nível 2 (Mediano) ⭐⭐* → Use só se você já ia comprar de qualquer forma.
• *Nível 3 (Bom) ⭐⭐⭐* → Bom para compras planejadas, mas ainda pode melhorar.
• *Nível 4 (Muito Bom) ⭐⭐⭐⭐* → Ótima promoção. Daqui pra cima já vale muito a pena!
• *Nível 5 (Excelente / Raro) ⭐⭐⭐⭐⭐* → Promoção rara, dessas que não aparecem toda hora. Aproveite sem medo!

Quanto maior o nível, melhor o custo-benefício para acumular milhas/pontos.

Ótimo dia a todos! Bora pra cima! 🚀
"""

## Grupo de Beleza e Cosméticos (07h am)

### Loja Natura

In [5]:
if hora_atual < hora_max_grupo_beleza:
  pagina = requests.get('https://www.comparemania.com.br/cashback-natura/')
  dados_pagina = BeautifulSoup(pagina.text, 'html.parser')

  #Extraindo os nomes dos clubes
  list_nomes_clubes = []
  nomes_clubes = dados_pagina.find_all('div', class_='d-grid')
  for div in nomes_clubes:
    texto_nome = div.find_all('div', class_='d-grid')
    for div in texto_nome:
      d_nome_clube = div.find('span').text
      if d_nome_clube == "Livelo":
        d_nome_clube = f'🟣 {d_nome_clube}'
      elif d_nome_clube == "Esfera":
        d_nome_clube = f'🔴 {d_nome_clube}'
      elif d_nome_clube == "Smiles":
        d_nome_clube = f'🟠 {d_nome_clube}'
      elif d_nome_clube in("Shopping Latam", "Azul"):
        d_nome_clube = f'🔵 {d_nome_clube}'

      list_nomes_clubes.append(d_nome_clube)
      print(d_nome_clube)


  #Extraindo a pontuacao dos clubes
  list_pontuacao_clubes = []
  pontuacao_clubes = dados_pagina.find_all('span', class_='text-center lh-70')

  for div in pontuacao_clubes:
    d_pontuacao_clube = div.text

    #Analise da pontuacao
    num_pontos = d_pontuacao_clube[:2]
    num_pontos = int(num_pontos.replace(",", ""))
    list_pontuacao_clubes.append(num_pontos)


  # Criando o DataFrame Unificado
  df_natura = pd.DataFrame({
      "nome_clube": list_nomes_clubes,
      "pontuacao_clube": list_pontuacao_clubes
  })

  list_analise_pontuacao = []

  for index, row in df_natura.iterrows():
      #print(index, row["nome_clube"], row["pontuacao_clube"])

      #Igualando os pesos entre pontos e milhas (considerando uma promo de 80% de bônus)
      if row["nome_clube"] not in("Livelo","Esfera"):
        pontuacao_ajustada = row["pontuacao_clube"] / 1.8
      else:
        pontuacao_ajustada = row["pontuacao_clube"]

      #Analisando a promoção
      if pontuacao_ajustada >= 12:
        d_analise = '⭐⭐⭐⭐⭐ Nível 5 (Excelente / Raro) 😏: Promoção rara, aproveite sem medo!'
      elif pontuacao_ajustada >= 8:
        d_analise = '⭐⭐⭐⭐ Nível 4 (Muito Bom) 😎: Ótima Promoção, daqui pra cima já vale muito!'
      elif pontuacao_ajustada >= 6:
        d_analise = '⭐⭐⭐ Nível 3 (Bom) 😉: Bom momento para potencializar compras planejadas, mas se puder aguardar, tem coisa melhor!'
      elif pontuacao_ajustada >= 3:
        d_analise = '⭐⭐ Nível 2 (Mediano) 🧐: Dá pra usar se você realmente já ia comprar, mas é melhor aguardar algo melhor ;)'
      else:
        d_analise = '⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor'

      list_analise_pontuacao.append(d_analise)


  df_natura['analise_pontuacao'] = list_analise_pontuacao
  df_natura['parceiro'] = 'Natura'
  df_natura['data_coleta_promocao'] = data_formatada
  df_natura

else:
  print("Fora do horário!")

🟠 Smiles
🔵 Azul
🟣 Livelo
🔵 Shopping Latam


### Loja Boticário

In [6]:
if hora_atual < hora_max_grupo_beleza:
  pagina = requests.get('https://www.comparemania.com.br/cashback-boticario/')
  dados_pagina = BeautifulSoup(pagina.text, 'html.parser')

  #Extraindo os nomes dos clubes
  list_nomes_clubes = []
  nomes_clubes = dados_pagina.find_all('div', class_='d-grid')
  for div in nomes_clubes:
    texto_nome = div.find_all('div', class_='d-grid')
    for div in texto_nome:
      d_nome_clube = div.find('span').text
      if d_nome_clube == "Livelo":
        d_nome_clube = f'🟣 {d_nome_clube}'
      elif d_nome_clube == "Esfera":
        d_nome_clube = f'🔴 {d_nome_clube}'
      elif d_nome_clube == "Smiles":
        d_nome_clube = f'🟠 {d_nome_clube}'
      elif d_nome_clube in("Shopping Latam", "Azul"):
        d_nome_clube = f'🔵 {d_nome_clube}'

      list_nomes_clubes.append(d_nome_clube)
      print(d_nome_clube)

  #Extraindo a pontuacao dos clubes
  list_pontuacao_clubes = []
  pontuacao_clubes = dados_pagina.find_all('span', class_='text-center lh-70')

  for div in pontuacao_clubes:
    d_pontuacao_clube = div.text

    #Analise da pontuacao
    num_pontos = d_pontuacao_clube[:2]
    num_pontos = int(num_pontos.replace(",", ""))
    list_pontuacao_clubes.append(num_pontos)


  df_boticario = pd.DataFrame({
      "nome_clube": list_nomes_clubes,
      "pontuacao_clube": list_pontuacao_clubes
  })

  list_analise_pontuacao = []

  for index, row in df_boticario.iterrows():
      #print(index, row["nome_clube"], row["pontuacao_clube"])

      #Igualando os pesos entre pontos e milhas (considerando uma promo de 80% de bônus)
      if row["nome_clube"] not in("Livelo","Esfera"):
        pontuacao_ajustada = row["pontuacao_clube"] / 1.8
      else:
        pontuacao_ajustada = row["pontuacao_clube"]

      #Analisando a promoção
      if pontuacao_ajustada >= 12:
        d_analise = '🟢 Nível 5 (Excelente / Raro) 🌟: Promoção rara, aproveite sem medo!'
      elif pontuacao_ajustada >= 8:
        d_analise = '🟢 Nível 4 (Muito Bom) 🔥: Ótima Promoção, daqui pra cima já vale muito!'
      elif pontuacao_ajustada >= 6:
        d_analise = '🟡 Nível 3 (Bom) ✨: Bom momento para potencializar compras planejadas, mas se puder aguardar, tem coisa melhor!'
      elif pontuacao_ajustada >= 3:
        d_analise = '🟡 Nível 2 (Mediano) ⭐: Dá pra usar se você realmente já ia comprar, mas é melhor aguardar algo melhor ;)'
      else:
        d_analise = '🔴 Nível 1 (Ruim) ⚠️: Sugiro aguardar algo melhor'

      list_analise_pontuacao.append(d_analise)


  df_boticario['analise_pontuacao'] = list_analise_pontuacao
  df_boticario['parceiro'] = 'Boticario'
  df_boticario['data_coleta_promocao'] = data_formatada
  df_boticario

else:
  print("Fora do horário!")

🟣 Livelo
🔴 Esfera


### Enviando os dados via Whatsapp

In [7]:
if hora_atual < hora_max_grupo_beleza:
  # Concatenando os dataframes
  df_final = pd.concat([df_natura, df_boticario], ignore_index=True)
  df_final

  # Criando a mensagem do grupo de Beleza e Cosméticos
  msg = ""
  msg_titulo = f"💄 *Promoções do Grupo de Beleza e Cosméticos em {data_formatada}: 👇*"

  for index, row in df_final.iterrows():
    msg += f"""
  *Parceiro:* {row["parceiro"]}
  *Programa de Fidelidade:* {row["nome_clube"]}
  *Pontuação:* {row["pontuacao_clube"]}
  *Análise da Promoção:* {row["analise_pontuacao"]}

  {"-"*30}"""
    #print(msg)
      #print(index, row["nome_clube"], row["pontuacao_clube"])

  msg_grupo = msg_titulo + "\n" + msg


  # Enviando mensagem no grupo do Whatsapp
  import requests

  # Substitua com os dados reais
  INSTANCE_ID = 'inst_4ng9578nwiz0'
  API_KEY = 'sk_hRzCfVl0hSFebdFmmN0hwhpNZtVA9JqIdscEpatVHyreMM9f'

  # JID do grupo (obtido via rota /groups)
  group_jid = '555195564436-1477420627@g.us'

  # Mensagem
  message_bom_dia = texto_bom_dia
  message_promo = msg_grupo

  # Endpoint
  url = f"https://api.zapperapi.com/{INSTANCE_ID}/messages/text"

  headers = {
      "X-Api-Key": API_KEY,
      "Content-Type": "application/json"
  }

  # Payload
  payload = {
      "jid": group_jid,
      "message": message_bom_dia,
  }

  #Enviando a mensagem de bom dia
  response = requests.post(url, json=payload, headers=headers)

  # Resposta
  if response.status_code == 200:
      print("Mensagem de bom dia enviada com sucesso para o grupo!")
  else:
      print(f"Erro ao enviar mensagem: {response.status_code} - {response.text}")


  # Aguardar 30 segundos
  import time
  time.sleep(30)

  # Payload
  payload = {
      "jid": group_jid,
      "message": message_promo,
  }

  #Enviando as promoções
  response = requests.post(url, json=payload, headers=headers)

  # Resposta
  if response.status_code == 200:
      print("Mensagem com as promoções enviada com sucesso para o grupo!")
  else:
      print(f"Erro ao enviar mensagem: {response.status_code} - {response.text}")

else:
  print("Fora do horário!")

Mensagem de bom dia enviada com sucesso para o grupo!
Mensagem com as promoções enviada com sucesso para o grupo!


##

## Grupo de Roupas e Calçados Esportivos (11h am)

### Loja Centauro

In [8]:
if hora_atual >= hora_min_grupo_esportivo:
  pagina = requests.get('https://www.comparemania.com.br/cashback-centauro/')
  dados_pagina = BeautifulSoup(pagina.text, 'html.parser')

  # Extraindo os nomes dos clubes
  list_nomes_clubes = []


  for sp in dados_pagina.select("div.d-grid span"):
      nome = sp.get_text(strip=True)

      # Normaliza para comparação (lowercase)
      nome_norm = nome.lower()

      # Apenas continua se for um dos nomes alvo
      if nome_norm not in nomes_alvo:
          continue

      # Aplica emoji conforme o programa
      if nome_norm == "livelo":
          nome = f"🟣 {nome}"
      elif nome_norm == "esfera":
          nome = f"🔴 {nome}"
      elif nome_norm == "smiles":
          nome = f"🟠 {nome}"
      elif nome_norm in ("azul", "shopping latam", "latam"):
          nome = f"🔵 {nome}"

      # Adiciona à lista final
      list_nomes_clubes.append(nome)
  list_nomes_clubes


  #Extraindo a pontuacao dos clubes
  list_pontuacao_clubes = []

  # percorre apenas a tabela "Pontos e Milhas"
  tabela_pm = dados_pagina.find('th', string=lambda s: s and 'Pontos e Milhas' in s)
  if tabela_pm:
      table = tabela_pm.find_parent('table')
      for tr in table.select('tbody tr'):
          # nome do programa (primeira coluna)
          nome_el = tr.select_one('td .d-grid span')
          if not nome_el:
              continue
          nome = nome_el.get_text(strip=True).lower()
          if nome not in nomes_alvo:
              continue

          # texto do ganho (segunda coluna)
          tds = tr.find_all('td')
          if len(tds) < 2:
              continue
          texto_ganho = tds[-1].get_text(" ", strip=True)  # exemplo: "1,00 pt/R$" ou "2,00 pt(s)"

          # extrai o primeiro número (1, 1,00, 4,5, 12, etc.) antes de "pt"
          m = re.search(r'(\d+(?:[.,]\d+)?)\s*pt', texto_ganho, flags=re.I)
          if not m:
              continue

          valor = m.group(1).replace(',', '.')  # normaliza decimal
          # Se você quer inteiro quando for 1,00, converta para float e depois para int se for inteiro exato:
          val_float = float(valor)
          val_final = int(val_float) if val_float.is_integer() else val_float

          list_pontuacao_clubes.append(val_final)




  # Criando o DataFrame
  df_centauro = pd.DataFrame({
      "nome_clube": list_nomes_clubes,
      "pontuacao_clube": list_pontuacao_clubes
  })

  list_analise_pontuacao = []

  for index, row in df_centauro.iterrows():
      #print(index, row["nome_clube"], row["pontuacao_clube"])

      #Igualando os pesos entre pontos e milhas (considerando uma promo de 80% de bônus)
      if row["nome_clube"] not in("Livelo","Esfera"):
        pontuacao_ajustada = row["pontuacao_clube"] / 1.8
      else:
        pontuacao_ajustada = row["pontuacao_clube"]

      #Analisando a promoção
      if pontuacao_ajustada >= 12:
        d_analise = '🟢 Nível 5 (Excelente / Raro) 🌟: Promoção rara, aproveite sem medo!'
      elif pontuacao_ajustada >= 8:
        d_analise = '🟢 Nível 4 (Muito Bom) 🔥: Ótima Promoção, daqui pra cima já vale muito!'
      elif pontuacao_ajustada >= 6:
        d_analise = '🟡 Nível 3 (Bom) ✨: Bom momento para potencializar compras planejadas, mas se puder aguardar, tem coisa melhor!'
      elif pontuacao_ajustada >= 3:
        d_analise = '🟡 Nível 2 (Mediano) ⭐: Dá pra usar se você realmente já ia comprar, mas é melhor aguardar algo melhor ;)'
      else:
        d_analise = '🔴 Nível 1 (Ruim) ⚠️: Sugiro aguardar algo melhor'

      list_analise_pontuacao.append(d_analise)


  df_centauro['analise_pontuacao'] = list_analise_pontuacao
  df_centauro['parceiro'] = 'Centauro'
  df_centauro['data_coleta_promocao'] = data_formatada
  display(df_centauro)

else:
  print("Fora do horário!")

Fora do horário!


### Loja NetShoes

In [9]:
if hora_atual >= hora_min_grupo_esportivo:
  pagina = requests.get('https://www.comparemania.com.br/cashback-netshoes/')
  dados_pagina = BeautifulSoup(pagina.text, 'html.parser')

  # Extraindo os nomes dos clubes
  list_nomes_clubes = []

  for sp in dados_pagina.select("div.d-grid span"):
      nome = sp.get_text(strip=True)

      # Normaliza para comparação (lowercase)
      nome_norm = nome.lower()

      # Apenas continua se for um dos nomes alvo
      if nome_norm not in nomes_alvo:
          continue

      # Aplica emoji conforme o programa
      if nome_norm in ("livelo", "shopping livelo"):
          nome = f"🟣 {nome}"
      elif nome_norm == "esfera":
          nome = f"🔴 {nome}"
      elif nome_norm == "smiles":
          nome = f"🟠 {nome}"
      elif nome_norm in ("azul", "shopping latam", "latam"):
          nome = f"🔵 {nome}"

      # Adiciona à lista final
      list_nomes_clubes.append(nome)
  list_nomes_clubes


  #Extraindo a pontuacao dos clubes
  list_pontuacao_clubes = []

  # percorre apenas a tabela "Pontos e Milhas"
  tabela_pm = dados_pagina.find('th', string=lambda s: s and 'Pontos e Milhas' in s)
  if tabela_pm:
      table = tabela_pm.find_parent('table')
      for tr in table.select('tbody tr'):
          # nome do programa (primeira coluna)
          nome_el = tr.select_one('td .d-grid span')
          if not nome_el:
              continue
          nome = nome_el.get_text(strip=True).lower()
          if nome not in nomes_alvo:
              continue

          # texto do ganho (segunda coluna)
          tds = tr.find_all('td')
          if len(tds) < 2:
              continue
          texto_ganho = tds[-1].get_text(" ", strip=True)  # exemplo: "1,00 pt/R$" ou "2,00 pt(s)"

          # extrai o primeiro número (1, 1,00, 4,5, 12, etc.) antes de "pt"
          m = re.search(r'(\d+(?:[.,]\d+)?)\s*pt', texto_ganho, flags=re.I)
          if not m:
              continue

          valor = m.group(1).replace(',', '.')  # normaliza decimal
          # Se você quer inteiro quando for 1,00, converta para float e depois para int se for inteiro exato:
          val_float = float(valor)
          val_final = int(val_float) if val_float.is_integer() else val_float

          list_pontuacao_clubes.append(val_final)




  # Criando o DataFrame
  df_netshoes = pd.DataFrame({
      "nome_clube": list_nomes_clubes,
      "pontuacao_clube": list_pontuacao_clubes
  })

  list_analise_pontuacao = []

  for index, row in df_netshoes.iterrows():
      #print(index, row["nome_clube"], row["pontuacao_clube"])

      #Igualando os pesos entre pontos e milhas (considerando uma promo de 80% de bônus)
      if row["nome_clube"] not in("Livelo","Esfera"):
        pontuacao_ajustada = row["pontuacao_clube"] / 1.8
      else:
        pontuacao_ajustada = row["pontuacao_clube"]

      #Analisando a promoção
      if pontuacao_ajustada >= 12:
        d_analise = '🟢 Nível 5 (Excelente / Raro) 🌟: Promoção rara, aproveite sem medo!'
      elif pontuacao_ajustada >= 8:
        d_analise = '🟢 Nível 4 (Muito Bom) 🔥: Ótima Promoção, daqui pra cima já vale muito!'
      elif pontuacao_ajustada >= 6:
        d_analise = '🟡 Nível 3 (Bom) ✨: Bom momento para potencializar compras planejadas, mas se puder aguardar, tem coisa melhor!'
      elif pontuacao_ajustada >= 3:
        d_analise = '🟡 Nível 2 (Mediano) ⭐: Dá pra usar se você realmente já ia comprar, mas é melhor aguardar algo melhor ;)'
      else:
        d_analise = '🔴 Nível 1 (Ruim) ⚠️: Sugiro aguardar algo melhor'

      list_analise_pontuacao.append(d_analise)


  df_netshoes['analise_pontuacao'] = list_analise_pontuacao
  df_netshoes['parceiro'] = 'Netshoes'
  df_netshoes['data_coleta_promocao'] = data_formatada
  display(df_netshoes)

else:
  print("Fora do horário!")

Fora do horário!


### Enviando os dados via Whatsapp

In [10]:
if hora_atual >= hora_min_grupo_esportivo:
  # Concatenando os dataframes
  df_final = pd.concat([df_centauro, df_netshoes], ignore_index=True)
  df_final

  # Criando a mensagem do grupo de Beleza e Cosméticos
  msg = ""
  msg_titulo = f"👟🏓 *Promoções do Grupo de Roupas e Calçados Esportivos em {data_formatada}: 👇*"

  for index, row in df_final.iterrows():
    msg += f"""
  *Parceiro:* {row["parceiro"]}
  *Programa de Fidelidade:* {row["nome_clube"]}
  *Pontuação:* {row["pontuacao_clube"]}
  *Análise da Promoção:* {row["analise_pontuacao"]}

  {"-"*30}"""
    #print(msg)
      #print(index, row["nome_clube"], row["pontuacao_clube"])

  msg_grupo = msg_titulo + "\n" + msg

  # Enviando mensagem no grupo do Whatsapp
  import requests

  # Substitua com os dados reais
  INSTANCE_ID = 'inst_4ng9578nwiz0'
  API_KEY = 'sk_hRzCfVl0hSFebdFmmN0hwhpNZtVA9JqIdscEpatVHyreMM9f'

  # JID do grupo (obtido via rota /groups)
  group_jid = '555195564436-1477420627@g.us'

  # Mensagem
  message_promo = msg_grupo

  # Endpoint
  url = f"https://api.zapperapi.com/{INSTANCE_ID}/messages/text"

  headers = {
      "X-Api-Key": API_KEY,
      "Content-Type": "application/json"
  }

  # Payload
  payload = {
      "jid": group_jid,
      "message": message_promo,
  }

  #Enviando as promoções
  response = requests.post(url, json=payload, headers=headers)

  # Resposta
  if response.status_code == 200:
      print("Mensagem com as promoções enviada com sucesso para o grupo!")
  else:
      print(f"Erro ao enviar mensagem: {response.status_code} - {response.text}")

else:
  print("Fora do horário!")


Fora do horário!


## ADD SEPHORA E EPOCA

## ADD AMAZON E MERCADO LIVRE

## ADD RENNER, C&A, RIACHUELO

In [11]:
### ADD MERCADO LIVRE, AMAZON, CASAS BAHIA, PONTO, MAGALU

## Mensagens com Imagens

## Jabar banco inter

In [12]:
import requests

# Substitua com os dados reais
INSTANCE_ID = 'inst_4ng9578nwiz0'
API_KEY = 'sk_hRzCfVl0hSFebdFmmN0hwhpNZtVA9JqIdscEpatVHyreMM9f'

# JID do grupo (obtido via rota /groups)
group_jid = '555195564436-1477420627@g.us'

# Mensagem
message_promo = f"""
Quando as pessoas me perguntam qual é um bom cartão para começar, eu sempre indico o cartão Inter Prime como uma opção,
pois ele oferece uma boa pontuação, pontos que nunca expiram e Sala VIP Grátis!

🟠 *Abra a sua conta no Banco Inter e ganhe 200 pontos*
Utilize o código 0F01D993 e abra a sua conta gratuita no Inter. Ao fazer isso, você já receberá 200 pontos Loop de presente!
👉 https://intergo.app/242d85c4

💰 * Quer pegar o cartão Inter Prime pagando menos? Use nosso cupom e economize R$30 no plano anual do Duo Gourmet:*
Utilize o código B1EE1054 para ganhar R$ 30 off na assinatura do plano anual pelo app do Inter!
👉 Acesse: https://intergo.app/1f148152

{"-"*30}

💳 Assista o vídeo abaixo para conhecer os principais benefícios do cartão Inter Prime 👇
https://www.youtube.com/playlist?list=PLywCMnijM298ruCE-Cey18WvsoV2UqqQf"""

# Imagem do Google Drive
drive_id = "1OpOK2Ee1q2yJO-UDhfP3w3hFh2uM7y0s"
img_url = f"https://drive.google.com/uc?export=view&id={drive_id}"

url = f"https://api.zapperapi.com/{INSTANCE_ID}/messages/media"

payload = {
    "jid": group_jid,
    "mediaType": "image",
    "mimetype": "image/jpeg",
    "media": img_url,
    "caption": message_promo,  # ← usa seu texto de análise
    "filename": "analise.jpg"
}

headers = {
    "X-Api-Key": API_KEY,
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

print(response.json())

{'status': 'scheduled', 'jobId': '42'}
